# Using the Analytics Engine (AE) to access station data
This notebook illustrates how to access the historical meteorological station data at stations of interest throughout WECC.

To execute a given 'cell' of this notebook, place the cursor in the cell and press the 'play' icon, or simply press shift+enter together. Some cells will take longer to run, and you will see a [$\ast$] to the left of the cell while AE is still working.

**Intended Application**: As a user, I want to **<span style="color:#FF0000">access the historical meteorological station data</span>** that is used to localize the dynamically downscaled WRF data in demand forecast modeling through:
1. Accessing and visualizing historical data at a station of interest
2. Exporting the historical data at a station of interest in a reader-friendly format

**Runtime**: With the default settings, this notebook takes approximately **less than 1 minute** to run from start to finish. Modifications to selections may increase the runtime. 

### Step 0: Set-up

In [ ]:
import climakitae as ck
import pandas as pd
import xarray as xr

from climakitae.util.utils import read_csv_file

### Step 1: Select a station of interest

First, we'll read in a station file of all the HadISD stations currently hooked up to the Analytics Engine. 

In [ ]:
# Import stations names and coordinates file
stations = "data/hadisd_stations.csv"
stations_df = read_csv_file(stations)
stations_df

The easiest way to access is to use the `ID` of the station. We'll demonstrate with "KSAC" for Sacramento Executive Airport. If you are looking to export multiple stations, the following cell is the only one you need to modify in this notebook. 

In [ ]:
my_station = 'KSAC' # set to any station of interest

In [ ]:
station_id = str(stations_df[stations_df['ID'] == my_station]['station id'].values[0])
filepaths = ["s3://cadcat/hadisd/HadISD_{}.zarr".format(s_id) for s_id in [station_id]]
filepaths # check the filepath

### Step 2: Retrieve the station data from the s3 bucket
Here we'll use a handy function to access the data. You don't have to change anything -- this step is set-up with the filepath we identified above.

In [ ]:
obs_ds = xr.open_mfdataset(
    filepaths,
    engine="zarr",
    consolidated=False,
    parallel=True,
    backend_kwargs=dict(storage_options={"anon": True}),
)

obs_ds

Next, we'll load the data into memory, which will help with both visualizing the data and exporting it in Step 3. 

In [ ]:
obs_ds = ck.load(obs_ds)

In [ ]:
ck.view(obs_ds)

### Step 3: Export the station into csv format for ease
To export into a reader-friendly format, we'll run the next cell. You don't need to modify anything here - it's set-up to automatically identify the station we have been using (KSAC, as the default). 

In [ ]:
filename = 'HadISD_{}'.format(station_id)
ck.export(obs_ds, filename, "csv")

### Step 4: Optional "fast access" mode
We provide this step where all of the above cells are condensed into a single run for faster access. We've set to a different station this time, to demonstrate. 

In [ ]:
# read in station data
stations = "data/hadisd_stations.csv"
stations_df = read_csv_file(stations)

# identify station to export
my_station = 'KACV' # set to any station of interst

# retrieves data
station_id = str(stations_df[stations_df['ID'] == my_station]['station id'].values[0])
filepaths = ["s3://cadcat/hadisd/HadISD_{}.zarr".format(s_id) for s_id in [station_id]]
obs_ds = xr.open_mfdataset(
    filepaths,
    engine="zarr",
    consolidated=False,
    parallel=True,
    backend_kwargs=dict(storage_options={"anon": True}),
)

# load data in for export
obs_ds = ck.load(obs_ds)

# export
filename = 'HadISD_{}'.format(station_id)
ck.export(obs_ds, filename, "csv")